In [1]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import LlamaCppEmbeddings
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks import StdOutCallbackHandler
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
import warnings
import torch
from langchain_chroma import Chroma


In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

callbacks = CallbackManager([StreamingStdOutCallbackHandler()])
handler = StdOutCallbackHandler()

embed_model_id = '/home/rust/Downloads/IBMC/NLP/embed/nomic-embed-text-v1.5'
model_kwargs = {'device': 'cuda', 'trust_remote_code' : True}
encode_kwargs = {'batch_size': 1, 'normalize_embeddings': True}

core_embeddings_model = HuggingFaceEmbeddings(model_name=embed_model_id, model_kwargs=model_kwargs, encode_kwargs = encode_kwargs)

persist_directory = "/home/rust/Downloads/IBMC/NLP/antiviral_rag/vectorstore_antiviral"
vectordb = Chroma(persist_directory=persist_directory, embedding_function=core_embeddings_model)

llm = LlamaCpp(
    model_path="/home/rust/Downloads/IBMC/NLP/models/llama-2-7b-chat.Q4_K_M.gguf",
    n_gpu_layers=-1,
    callbacks=callbacks,
    f16_kv=True,
    n_ctx=4096,
    n_batch=512,
    verbose=True,
    temperature=0
)

prompt_template = """Context information is below.
---------------------
{context}
---------------------
You are an expert in medicinal chemistry and pharmacology. Given the context information, answer the query. Follow a set of rules listed below:
1. Don't make up a source. If you can't find the source, just write "I can't find the source'"
2. If you find the answer, write the **BRIEF** and **CONCISE** answer and **ALWAYS** return the list of sources in the form 'https://pubmed.ncbi.nlm.nih.gov/source'.
Query: {question}
Answer: """

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_template) 

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, callbacks=None, verbose=True)

vectorstore_retriever = vectordb.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.7},
    k=20,
)

#ensemble_retriever = EnsembleRetriever(
#    retrievers=[vectorstore_retriever, keyword_retriever],
#    weights=[0.4, 0.6]
#)

compressor = FlashrankRerank(top_n = 4)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vectorstore_retriever
)


document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Content:{page_content}\nSource:https://pubmed.ncbi.nlm.nih.gov/{source}\n",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=compression_retriever,
    verbose=True,
    return_source_documents=True
)


#print("Chatbot initialized, ready to chat...")
#while True:
#    question = input("> ")
#    answer = qa(question) 


In [ ]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import LlamaCppEmbeddings
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks import StdOutCallbackHandler
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
import warnings
import torch
from langchain_chroma import Chroma


torch.cuda.empty_cache()
device = torch.device("cuda")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

callbacks = CallbackManager([StreamingStdOutCallbackHandler()])
handler = StdOutCallbackHandler()

llm = LlamaCpp(
    model_path="/home/rust/Downloads/IBMC/NLP/models/llama-2-7b-chat.Q4_K_M.gguf",
    n_gpu_layers=-1,
    callbacks=callbacks,
    f16_kv=True,
    n_ctx=4096,
    n_batch=512,
    verbose=True,
    temperature=0
)

embed_model_id = '/home/rust/Downloads/IBMC/NLP/embed/nomic-embed-text-v1.5'
model_kwargs = {'device': 'cuda', 'trust_remote_code' : True}
encode_kwargs = {'batch_size': 1, 'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(model_name=embed_model_id, model_kwargs=model_kwargs, encode_kwargs = encode_kwargs)

persist_directory = "/home/rust/Downloads/IBMC/NLP/antiviral_rag/vectorstore_antiviral"
vectordb = Chroma(persist_directory=persist_directory, embedding_function=core_embeddings_model)



In [ ]:
import pandas as pd
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset 
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas import evaluate

callbacks = CallbackManager([StreamingStdOutCallbackHandler()])
handler = StdOutCallbackHandler()

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'], 
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}

llm = LlamaCpp(
    model_path="/home/rust/Downloads/IBMC/NLP/models/llama-2-7b-chat.Q4_K_M.gguf",
    n_gpu_layers=-1,
    callbacks=callbacks,
    f16_kv=True,
    n_ctx=4096,
    n_batch=512,
    verbose=True,
    temperature=0
)

llm = LangchainLLMWrapper(llm)


embed_model_id = '/home/rust/Downloads/IBMC/NLP/embed/nomic-embed-text-v1.5'
model_kwargs = {'device': 'cuda', 'trust_remote_code' : True}
encode_kwargs = {'batch_size': 1, 'normalize_embeddings': True}

embedding_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs=model_kwargs, 
    encode_kwargs = encode_kwargs
)

embedding_model = LangchainEmbeddingsWrapper(embedding_model)

dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[faithfulness, answer_relevancy, context_precision, context_recall],llm=llm,embeddings=embedding_model)
score.to_pandas()
score.to_csv('./')


/home/rust/Downloads/IBMC/NLP/cuda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/rust/Downloads/IBMC/NLP/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loade

.......AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
...........................................................................................
llama_new_context_with_model: kv self size  = 2048.00 MB
llama_new_context_with_model: compute buffer total size =  281.47 MB
<All keys matched successfully>
Evaluating:   0%|                                                                                 | 0/8 [00:00<?, ?it/s]

In [1]:
import pandas as pd
from deepeval.integrations.llama_index import DeepEvalFaithfulnessEvaluator
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from deepeval.integrations import trace_llama_index
from llama_index.llms.ollama import Ollama
from langchain_community.llms import LlamaCpp
from llama_index.llms.langchain import LangChainLLM
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks import StdOutCallbackHandler
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import warnings
import torch

OPENAI_API_KEY="1234"

torch.cuda.empty_cache()
device = torch.device("cuda")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

callbacks = CallbackManager([StreamingStdOutCallbackHandler()])
handler = StdOutCallbackHandler()

trace_llama_index(auto_eval=True)

#llm = LlamaCpp(
#    model_path="/home/rust/Downloads/IBMC/NLP/models/llama-2-7b-chat.Q4_K_M.gguf",
#    n_gpu_layers=25,
#    callbacks=callbacks,
#    f16_kv=True,
#    n_ctx=4096,
#    n_batch=512,
#    verbose=True,
#    temperature=0
#) 

Settings.llm = Ollama(model="mistral:latest", request_timeout=120.0)

model_name = '/home/rust/Downloads/IBMC/NLP/embed/nomic-embed-text-v1.5'

Settings.embed_model = HuggingFaceEmbedding(model_name=model_name, trust_remote_code=True)

documents = SimpleDirectoryReader("./deepeval_test").load_data()
index = VectorStoreIndex.from_documents(documents)
rag_application = index.as_query_engine()

user_input = "When was the first super bowl?"
    

response_object = rag_application.query(user_input)

evaluator = DeepEvalFaithfulnessEvaluator()
evaluation_result = evaluator.evaluate_response(
    query=user_input, response=response_object
)
print(evaluation_result)



/home/rust/Downloads/IBMC/NLP/cuda/lib/python3.10/site-packages/deepeval/__init__.py:53: UserWarning: You are using deepeval version 2.3.7, however version 2.3.8 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(
/home/rust/Downloads/IBMC/NLP/cuda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.